# Estimate surface volume under a shape layer
<font color='blue'>The following code calculates volume of a DEM under a specified elevation. The DEM is extracted from a larger DEM by clipping with a shapefile. Specific python libraries used to carryout the analysis are *<font color='red'>numpy</font> (for numerical calculation)*, *<font color='red'>matplotlib</font> (for visualization)*, *<font color='red'>rioxarray</font> (for raster analysis)*, *<font color='red'>geopandas</font> (for handling of shapefiles)* and *<font color='red'>pandas</font> (to save a result if required)*.</font>

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import rioxarray as rxr

In [ ]:
# Define a couple of utility function
def base_height_est(elevation_data, method = 1):
    """
    Returns the base height as well as the data array
    
    method 1 calulates the median value of all the boundary pixels and set that 
    as the base height under which the volume is calculated
    
    method 2 assigns the 95th pernetile height value as he base height and also changes 
    the original data set by removing the outlier pixels (having extreme high and 
                                                          low elevation values)

    method 3 directly assigns the median value of the raw elevation data as the base_height

    method 4 assigns the maximum value of the raw elevation data as the base height

    ** method 3 and 4 only should be used in case of good quality elevation data 
    """
    from skimage.segmentation import find_boundaries
    import numpy as np
    
    data = elevation_data
    
    if method == 1:
        
        data_copy = data.copy()
        data_copy[np.isnan(data)] = 0
        data_copy[~np.isnan(data)] = 1

        boundary = find_boundaries(data_copy, connectivity=1, mode='inner', background=0)
        bound_pixels = data * boundary
        bound_pixels[bound_pixels == 0] = np.nan
        base_height = np.nanpercentile(bound_pixels, 50)
    
    elif method == 2:
        
        data_c = data.copy()
        data_c[(data < np.nanpercentile(data, 5)) | (data > np.nanpercentile(data, 95))] = np.nan
        base_height = np.nanmax(data_c)
        data = data_c

    elif method == 3:
        base_height = np.nanpercentile(data, 50)

    elif method == 4:
        base_height = np.nanmax(data)
    
    return base_height, data

def plot_DEM_3D(rio_dem):
    """
    plot data array extracted from a rioxarray dem (rio_dem) in 3D. Uses mpl_toolkits library to create 3D axes.
    """
    # rio_dem is a rioxarray data set
    from mpl_toolkits import mplot3d
    import numpy as np
    
    data = rio_dem.data[0].astype('float')
    data[(data < -9000) | (data > 9000)] = np.nan
    
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    X, Y = np.meshgrid(rio_dem.x, rio_dem.y)
    ax.plot_surface(X,Y,data, cmap='viridis', antialiased=False)

In [ ]:
help(base_height_est)

In [ ]:
DEM_in = "dem_bangalore_43N.tif" # This tiff file is a local variable. It has to be uploaded in the local repository before running the code
dem = rxr.open_rasterio(DEM_in)

In [ ]:
dem

In [ ]:
feat_in = "polygon_43N.shp" # This shapefile is a local variable. Along with the .shp file, all other files related to the specific shapefile has to be stored in the local repository
gdf = gpd.read_file(feat_in)

In [ ]:
gdf

In [ ]:
# Main line of codes. Check for the method argument used in the base_height_est function inside the for loop. Default is 1
# We need a crs (coordinate refrence system) to be assigned to each feature layers
crs_sub = "epsg:" + str(gdf.crs.to_epsg())
# To save the output, i.e. 3D surface volume cooresponding to each of the feature layers, a dataframe is to be created
df = pd.DataFrame(columns = ["feat_ID", "volume (in cubic m)"])

for i in range(len(gdf)):
    gdf_sub = gpd.GeoDataFrame(index = [i], 
                           crs = crs_sub, 
                           geometry = [gdf.iloc[i].geometry])
    dem_clip = dem.rio.clip(gdf_sub.geometry)
    # If we need to save the clipped file . To save the rioxrray file
    # dem_clip.rio.to_raster("dem01_projected_clip01.tif")
    
    # Calculation of the surface volume below certain height
    data = dem_clip.data[0].astype('float')
    data[(data < -9000) | (data > 9000)] = np.nan # -/+9000 m some random number. Mostly to filter out the void values
    base_height, data = base_height_est(data, method = 1) # see the function "base_height_est"
    pixel_area = dem_clip.rio.resolution()[0] ** 2
    surface_volume = np.nansum((data < base_height) * (base_height - data) * pixel_area) # in cubic meter
    # store the result
    df.loc[i, "feat_ID"] = gdf.loc[i, "id"]
    df.loc[i, "volume (in cubic m)"] = surface_volume
    print(i)

# If you want to see a clipped DEM in 3D
# plot_DEM_3D(dem_clip)

In [ ]:
df

In [ ]:
# To save the dataframe with result as xlsx file
df.to_csv("surface_volume.csv", index = False) # The .csv file gets saved in a local repo inside "mybinder". That has to be downloaded in order to save

In [1]:
%load_ext watermark

%watermark -v -m -p numpy,matplotlib,geopandas,pandas,rioxarray,scikit-image,watermark

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.24.0

numpy       : 1.26.4
matplotlib  : 3.8.4
geopandas   : 0.14.4
pandas      : 2.2.1
rioxarray   : 0.15.5
scikit-image: not installed
watermark   : 2.4.3

Compiler    : MSC v.1938 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : Intel64 Family 6 Model 183 Stepping 1, GenuineIntel
CPU cores   : 32
Architecture: 64bit

